In [ ]:
import pandas as pd
import numpy as np
import scanpy as sc
from matplotlib.pyplot import rc_context
import scanpy.external as sce
import seaborn as sns
import matplotlib.pyplot as plt
from statannot import add_stat_annotation
from tqdm import tqdm_notebook
import warnings
from scipy.stats import ranksums
import matplotlib
warnings.filterwarnings("ignore")
from scipy.spatial import KDTree
import os

In [ ]:
sns.__version__

In [ ]:
def remove_filled_color(ax):
    colors = []
    for collection in ax.collections:
        if isinstance(collection, matplotlib.collections.PolyCollection):
            colors.append(collection.get_facecolor())
            collection.set_edgecolor(colors[-1])
            collection.set_facecolor('none')
    if len(ax.lines) == 2 * len(colors):  # suppose inner=='box'
        for lin1, lin2, color in zip(ax.lines[::2], ax.lines[1::2], colors):
            lin1.set_color(color)
            lin2.set_color(color)
    for h in ax.legend_.legendHandles:
        if isinstance(h, matplotlib.patches.Rectangle):
            h.set_edgecolor(h.get_facecolor())
            h.set_facecolor('none')
            h.set_linewidth(1.5)
    return ax

In [ ]:
list1 = []
list2 = []
list3 = []
list4 = []
file_names = os.listdir('E:\\spatial interactions\\data_store\\')
for ele in file_names:
    adata = sc.read_h5ad(r'E:\\spatial interactions\\data_store\\'+ele)
    leiden_list = list(adata.obs['leiden2'])
    leiden_list = ['malignant' if x == 'Malignant' else x for x in leiden_list]
    adata.obs['leiden2'] = leiden_list
    data = pd.read_excel(r'E:\\stemness genes\\pnas.1818210116.sd01.xlsx')
    stemness_genes = list(data.iloc[:,0].dropna())
    stemness_genes_interaction = np.intersect1d(stemness_genes, adata.var.index)
    adata.obs['stemnness'] = adata[:,stemness_genes_interaction].X.mean(axis=1)
    
    spatial_coords = pd.DataFrame(adata.obsm['spatial'], index = adata.obs.index)
    cell_labels = adata.obs['leiden2']
    malignant_indices = cell_labels[cell_labels == 'malignant'].index
    other_CAF_indices = cell_labels[cell_labels == 'other CAF'].index

    tree = KDTree(spatial_coords)
    nearest_neighbors = tree.query(spatial_coords.loc[malignant_indices], k=7)
    nearest_neighbor_indices = nearest_neighbors[1][:, 1]
    nearest_neighbor_labels = cell_labels.iloc[nearest_neighbor_indices]
    malignant_with_other_CAF_neighbors = malignant_indices[nearest_neighbor_labels == 'other CAF']
    
    MCAM_CAF_indices = cell_labels[cell_labels == 'MCAM CAF'].index
    tree = KDTree(spatial_coords)
    nearest_neighbors = tree.query(spatial_coords.loc[malignant_indices], k=7)
    nearest_neighbor_indices = nearest_neighbors[1][:, 1]
    nearest_neighbor_labels = cell_labels.iloc[nearest_neighbor_indices]
    malignant_with_MCAM_CAF_neighbors = malignant_indices[nearest_neighbor_labels == 'MCAM CAF']
    
    adata1 = adata[malignant_with_other_CAF_neighbors,]
    adata2 = adata[malignant_with_MCAM_CAF_neighbors,]
    other_malignant = list(set(malignant_indices)-set(malignant_with_other_CAF_neighbors))
    adata3 = adata[other_malignant,]
    other_malignant = list(set(malignant_indices)-set(malignant_with_MCAM_CAF_neighbors))
    adata4 = adata[other_malignant,]
    if list(adata1.obs['stemnness']):
        list1.extend(list(adata1.obs['stemnness']))
    if list(adata2.obs['stemnness']):
        list2.extend(list(adata2.obs['stemnness']))
    if list(adata3.obs['stemnness']):
        list3.extend(list(adata3.obs['stemnness']))
    if list(adata4.obs['stemnness']):
        list4.extend(list(adata4.obs['stemnness']))
    print(ele, np.mean(adata1.obs['stemnness']), np.mean(adata2.obs['stemnness']))

In [ ]:
print(np.mean(list1), np.mean(list2), np.mean(list3), np.mean(list4))

In [ ]:
ranksums(list1, list3)

In [ ]:
ranksums(list1, list2)

In [ ]:
ranksums(list4, list2)

In [ ]:
ranksums(list3, list2)

In [ ]:
data_temp = pd.DataFrame({
    "score": list2 + list4+list1 + list3,
    "type": ['P'] * len(list2) + ['R'] * len(list4)+['P1'] * len(list1) + ['R1'] * len(list3)
})
data_temp[''] = 'stemness'

In [ ]:
my_pal ={'P':'#1f77b4','R':'#b1c2c9','P1':'#d62627','R1':'#bfa6a6'}
sns.set_theme(style="white",font='Arial',font_scale=1.9)
fig, axes = plt.subplots(1,1, figsize=(2,1.4))
ax = sns.violinplot(x="", y="score", hue="type", data=data_temp, palette=my_pal,  saturation=1)
ax.set(yticklabels=[])
ax = remove_filled_color(ax)
pairs = [('P','R')]
ax.legend_.remove()
ax.set_ylabel('')
plt.yticks(rotation=0)
ax.tick_params(axis='both', which='major', labelbottom = False, bottom=False, top = False, labeltop=False)
ax.spines[['right', 'top']].set_visible(False)
ax.yaxis.set_ticks_position('left')
#ax.set_xticks([-0.2,0.2])
#ax.xaxis.set_ticks_position('bottom')###这是在最后一行加上两个ticks
lines, labels = fig.axes[-1].get_legend_handles_labels()    
ax.set(xticklabels=[])
ax.set_ylim(-0.05, 1.2)  # Adjust limits to add space
ax.set_yticks([-0.2,0.5])
plt.tight_layout(pad=0.08,h_pad=0.5,w_pad=1)
plt.savefig('stemnness_near_MCAM.svg', bbox_inches="tight")
plt.show()